In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
import gmaps
import os
ABS_path = "Data/ABS_Labourstatus_Data.csv"
ABS_metadata = pd.read_csv(ABS_path, low_memory = False)
ABS_metadata

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value,Flag Codes,Flags
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,236.0,NaN,NaN
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,113.0,NaN,NaN
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,348.0,NaN,NaN
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,105.0,NaN,NaN
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,58.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158545,T55,55 - 64,1,Males,4,"Employed, hours of work not stated",1,New South Wales,LGA2016,Local Government Areas (2016),11100,Botany Bay (C),2016.0,2016.0,32.0,NaN,NaN
158546,T45,45 - 54,2,Females,TOT,Total,3,Queensland,LGA2016,Local Government Areas (2016),33830,Hope Vale (S),2016.0,2016.0,45.0,NaN,NaN
158547,T25,25 - 34,2,Females,4,"Employed, hours of work not stated",5,Western Australia,LGA2016,Local Government Areas (2016),57000,Perenjori (S),2016.0,2016.0,0.0,NaN,NaN
158548,T75,75 - 84,1,Males,LF,Total Labour Force,1,New South Wales,LGA2016,Local Government Areas (2016),12930,Georges River (A),2016.0,2016.0,167.0,NaN,NaN


In [4]:
#Remove Flag Code and Flags Colums
ABS_metadata = ABS_metadata.drop(['Flag Codes', 'Flags'], axis=1)
#Remove NaN rows
ABS_metadata = ABS_metadata.dropna()
ABS_metadata

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,236.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,113.0
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,348.0
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,105.0
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158544,T55,55 - 64,1,Males,LF,Total Labour Force,5,Western Australia,LGA2016,Local Government Areas (2016),58400,Trayning (S),2016.0,2016.0,22.0
158545,T55,55 - 64,1,Males,4,"Employed, hours of work not stated",1,New South Wales,LGA2016,Local Government Areas (2016),11100,Botany Bay (C),2016.0,2016.0,32.0
158546,T45,45 - 54,2,Females,TOT,Total,3,Queensland,LGA2016,Local Government Areas (2016),33830,Hope Vale (S),2016.0,2016.0,45.0
158547,T25,25 - 34,2,Females,4,"Employed, hours of work not stated",5,Western Australia,LGA2016,Local Government Areas (2016),57000,Perenjori (S),2016.0,2016.0,0.0


In [5]:
new = ABS_metadata['Region'].str.split('(', n = 1, expand = True)
ABS_metadata['Region'] = new[0]
ABS_metadata

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,236.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,113.0
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,348.0
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,105.0
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158544,T55,55 - 64,1,Males,LF,Total Labour Force,5,Western Australia,LGA2016,Local Government Areas (2016),58400,Trayning,2016.0,2016.0,22.0
158545,T55,55 - 64,1,Males,4,"Employed, hours of work not stated",1,New South Wales,LGA2016,Local Government Areas (2016),11100,Botany Bay,2016.0,2016.0,32.0
158546,T45,45 - 54,2,Females,TOT,Total,3,Queensland,LGA2016,Local Government Areas (2016),33830,Hope Vale,2016.0,2016.0,45.0
158547,T25,25 - 34,2,Females,4,"Employed, hours of work not stated",5,Western Australia,LGA2016,Local Government Areas (2016),57000,Perenjori,2016.0,2016.0,0.0


In [6]:
#State just WA
ABS_WA_data = ABS_metadata.set_index('STATE')

In [7]:
ABS_WA_data = ABS_WA_data.drop([1,2,3,4,6,7,8,9], inplace=False)

In [8]:
ABS_WA_data = ABS_WA_data.reset_index()
ABS_WA_data

,STATE,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
0,5,A15,15 - 19,1,Males,1,"Employed, worked full-time",Western Australia,LGA2016,Local Government Areas (2016),50080,Albany,2016.0,2016.0,204.0
1,5,A15,15 - 19,2,Females,1,"Employed, worked full-time",Western Australia,LGA2016,Local Government Areas (2016),50080,Albany,2016.0,2016.0,101.0
2,5,A15,15 - 19,3,Persons,1,"Employed, worked full-time",Western Australia,LGA2016,Local Government Areas (2016),50080,Albany,2016.0,2016.0,302.0
3,5,A15,15 - 19,1,Males,1,"Employed, worked full-time",Western Australia,LGA2016,Local Government Areas (2016),50210,Armadale,2016.0,2016.0,280.0
4,5,A15,15 - 19,2,Females,1,"Employed, worked full-time",Western Australia,LGA2016,Local Government Areas (2016),50210,Armadale,2016.0,2016.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39207,5,T65,65 - 74,1,Males,4,"Employed, hours of work not stated",Western Australia,LGA2016,Local Government Areas (2016),53640,Gnowangerup,2016.0,2016.0,0.0
39208,5,T75,75 - 84,2,Females,UEMP,Total Unemployed,Western Australia,LGA2016,Local Government Areas (2016),52310,Cuballing,2016.0,2016.0,0.0
39209,5,T65,65 - 74,2,Females,6,"Unemployed, looking for part-time work",Western Australia,LGA2016,Local Government Areas (2016),57080,Perth,2016.0,2016.0,5.0
39210,5,T55,55 - 64,1,Males,LF,Total Labour Force,Western Australia,LGA2016,Local Government Areas (2016),58400,Trayning,2016.0,2016.0,22.0


In [12]:
#import region/district
path_region_district= 'Data/region_by_district_wa.csv'
region_district = pd.read_csv(path_region_district)
region_district.head()
#income_by_district=pd.merge(wa_mean_income_by_LGA,region_district,left_on=‘Region’,right_on=‘Region’)

,Region,District
0,Albany,Great Southern District
1,Armadale,Armadale District
2,Ashburton,Pilbara District
3,Augusta-Margaret River,South West District
4,Bassendean,Midland District


In [14]:
Disctrict_pops = ABS_WA_data.loc[(ABS_WA_data["Labour force status"] == "Total")]
Disctrict_pops = Disctrict_pops.groupby("Region").sum()
Disctrict_pops = Disctrict_pops.drop(['STATE', 'SEX_ABS','TIME',"Census year"], axis=1)
Disctrict_pops = Disctrict_pops.rename(columns = {"Value":"Total Persons"})
Disctrict_pops

,Total Persons
Region,
Albany,106766.0
Armadale,200914.0
Ashburton,40444.0
Augusta-Margaret River,40123.0
Bassendean,43305.0
...,...
Wyalkatchem,1512.0
Wyndham-East Kimberley,20716.0
Yalgoo,1062.0


In [16]:
pop_by_district=pd.merge(Disctrict_pops,region_district,left_on='Region',right_on='Region')
pop_by_district

,Region,Total Persons,District
0,Albany,106766.0,Great Southern District
1,Armadale,200914.0,Armadale District
2,Ashburton,40444.0,Pilbara District
3,Augusta-Margaret River,40123.0,South West District
4,Bassendean,43305.0,Midland District
...,...,...,...
134,Wyalkatchem,1512.0,Wheatbelt District
135,Wyndham-East Kimberley,20716.0,Kimberley District
136,Yalgoo,1062.0,Mid West-Gascoyne District
137,Yilgarn,3642.0,Wheatbelt District


In [17]:
Disctrict_pops = pop_by_district.groupby("District").sum()

In [18]:
Disctrict_pops

,Total Persons
District,
Armadale District,578341.0
Cannington District,625913.0
Fremantle District,744304.0
Goldfields-Esperance Region,146444.0
Great Southern District,222989.0
Joondalup District,953314.0
Kimberley District,91507.0
Mandurah District,685810.0
Mid West-Gascoyne District,170683.0


In [19]:
Disctrict_pops.to_csv("District_Population.csv")